# 회원정보 전처리 (1차)

In [27]:
import os
import pandas as pd
import gc
import numpy as np

os.chdir("C:/Users/Money/Documents/final_project")
df1회원정보 = pd.read_parquet("1.회원정보_all_month.parquet")
df1 = df1회원정보.drop('Segment', axis=1)
y = df1회원정보['Segment']

In [28]:
# 결측처리 (범주형)
# 범주형 데이터만 추출해 리스트에 넣음
categorical_cols = df1.select_dtypes(include=['object']).columns.tolist()
print(categorical_cols)

['ID', '연령', '가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']


In [29]:
# 범주형 데이터 중 결측치가 존재하는 행만 재할당
categorical_with_na = [col for col in categorical_cols if df1[col].isnull().any()]
print(categorical_with_na)

['가입통신회사코드', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분']


In [30]:
# 결측치 '기타'로 대체
df1[categorical_with_na] = df1[categorical_with_na].fillna('기타')

In [31]:
# 결측처리 (수치형)
# 수치형 데이터(int, float)만 추출해 리스트에 넣음
numeric_cols = df1.select_dtypes(include=['number']).columns.tolist()
print(numeric_cols)

['기준년월', '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월']


In [32]:
# 수치형 데이터 중 결측치가 존재하는 행만 재할당
numeric_cols_na = [col for col in numeric_cols if df1[col].isnull().any()]
print(numeric_cols_na)

['최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자']


In [33]:
# 결측치 각 행의 평균으로 대체
for col in numeric_cols_na:
    df1[col].fillna(df1[col].mean(), inplace=True)

In [34]:
# 수치형 0, 문자형 0, 문자형 0개 인 값으로만 이뤄진 컬럼 제거
allowed_values = {0, '0', '0개'}

df1_0_columns = df1.columns[
    df1.apply(lambda col: set(col.unique()).issubset(allowed_values))
].tolist()

# print(df1_0_columns)

df1_filtered = df1.drop(columns=df1_0_columns)

# total df
print(df1_filtered)

          기준년월            ID  남녀구분코드     연령  회원여부_이용가능  회원여부_이용가능_CA  \
0       201807  TRAIN_000000       2    40대          1             1   
1       201807  TRAIN_000001       1    30대          1             1   
2       201807  TRAIN_000002       1    30대          1             1   
3       201807  TRAIN_000003       2    40대          1             1   
4       201807  TRAIN_000004       2    40대          1             1   
...        ...           ...     ...    ...        ...           ...   
399995  201812  TRAIN_399995       2  70대이상          1             1   
399996  201812  TRAIN_399996       2    50대          1             1   
399997  201812  TRAIN_399997       1    30대          1             1   
399998  201812  TRAIN_399998       1    40대          1             1   
399999  201812  TRAIN_399999       2    40대          1             1   

        회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  소지카드수_이용가능_신용  ...  \
0                   0        1            1              1  ...   
1

In [35]:
df1.isna().sum()

기준년월             0
ID               0
남녀구분코드           0
연령               0
회원여부_이용가능        0
                ..
우수회원면제카드수_B0M    0
기타면제카드수_B0M      0
카드신청건수           0
Life_Stage       0
최종카드발급경과월        0
Length: 77, dtype: int64

In [ ]:
df1.shape

,기준년월,ID,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,1,1,0,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,1,1,0,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,1,1,0,1,2,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15
